# Comparing Optimizers in TensorFlow ANN
This notebook trains the same ANN model using different optimizers and compares their accuracy.

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

# Load dataset
dataset = pd.read_csv('/content/cell_data.csv')

# Display dataset info
dataset.info()
print(dataset.head())

# Replace '?' with NaN and drop missing values
dataset.replace('?', np.nan, inplace=True)
dataset.dropna(inplace=True)

# Convert 'BareNuc' column to numeric
dataset['BareNuc'] = dataset['BareNuc'].astype(int)

# Rename 'Temp Class' column to 'Class' to avoid issues
dataset.rename(columns={'Temp Class': 'Class'}, inplace=True)

# Separate features and target variable
X = dataset.iloc[:, 1:-1].values  # Exclude ID column and Class column
y = dataset.iloc[:, -1].values  # Class column is the target

# Encode target labels (assuming binary classification: 0 & 1)
le = LabelEncoder()
y = le.fit_transform(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Feature scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Define optimizers to test
optimizers = {
    'SGD': tf.keras.optimizers.SGD(),
    'RMSprop': tf.keras.optimizers.RMSprop(),
    'Adagrad': tf.keras.optimizers.Adagrad(),
    'Adadelta': tf.keras.optimizers.Adadelta(),
    'Nadam': tf.keras.optimizers.Nadam(),
    'FTRL': tf.keras.optimizers.Ftrl(),
    'Adam': tf.keras.optimizers.Adam()
}

# Store results
results = []

for name, optimizer in optimizers.items():
    # Initialize ANN
    ann = tf.keras.models.Sequential()
    ann.add(tf.keras.layers.Dense(units=6, activation='relu', input_shape=(X_train.shape[1],)))
    ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
    ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

    # Compile the ANN
    ann.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Train the ANN
    ann.fit(X_train, y_train, batch_size=32, epochs=100, verbose=0)

    # Make predictions
    y_pred = (ann.predict(X_test) > 0.5)

    # Evaluate model performance
    acc = accuracy_score(y_test, y_pred)
    print(f"\n{name} Optimizer")
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("Accuracy Score:", acc)

    results.append((name, acc * 100))

# Display all results as a DataFrame
results_df = pd.DataFrame(results, columns=['Optimizer', 'Accuracy (%)'])
print("\nFinal Accuracy Table:")
print(results_df.sort_values(by='Accuracy (%)', ascending=False).reset_index(drop=True))

# Predict a single sample (example input, should be replaced with real data)
sample_input = sc.transform([[5, 1, 1, 1, 2, 1, 3, 1, 1]])
print("\nSingle Sample Prediction (Adam):", ann.predict(sample_input) > 0.5)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           699 non-null    int64 
 1   Clump        699 non-null    int64 
 2   UnifSize     699 non-null    int64 
 3   UnifShape    699 non-null    int64 
 4   MargAdh      699 non-null    int64 
 5   SingEpiSize  699 non-null    int64 
 6   BareNuc      699 non-null    object
 7   BlandChrom   699 non-null    int64 
 8   NormNucl     699 non-null    int64 
 9   Mit          699 non-null    int64 
 10  Temp Class   699 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 60.2+ KB
        ID  Clump  UnifSize  UnifShape  MargAdh  SingEpiSize BareNuc  \
0  1000025      5         1          1        1            2       1   
1  1002945      5         4          4        5            7      10   
2  1015425      3         1          1        1            2       2   
3  1016277      6     

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 

SGD Optimizer
Confusion Matrix:
 [[125   5]
 [  5  70]]
Accuracy Score: 0.9512195121951219


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 

RMSprop Optimizer
Confusion Matrix:
 [[125   5]
 [  3  72]]
Accuracy Score: 0.9609756097560975


...

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step

Nadam Optimizer
Confusion Matrix:
 [[126   4]
 [  5  70]]
Accuracy Score: 0.9560975609756097


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 

FTRL Optimizer
Confusion Matrix:
 [[130   0]
 [ 75   0]]
Accuracy Score: 0.6341463414634146


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 

Adam Optimizer
Confusion Matrix:
 [[126   4]
 [  4  71]]
Accuracy Score: 0.9609756097560975

Final Accuracy Table:
  Optimizer  Accuracy (%)
0   RMSprop     96.097561
1      Adam     96.097561
2     Nadam     95.609756
3       SGD     95.121951
4   Adagrad     92.195122
5  Adadelta     74.146341
6      FTRL     63.414634
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step

Single Sample Prediction (Adam): [[False]]
